# Clinical Trial Success Prediction

In [15]:
# Load packages and data
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import shap
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
import os

# Load AACT tables
studies = pd.read_csv("clinical_data/studies.txt", sep="|", low_memory=False)
sponsors = pd.read_csv("clinical_data/sponsors.txt", sep="|", low_memory=False)
interventions = pd.read_csv("clinical_data/interventions.txt", sep="|", low_memory=False)
designs = pd.read_csv("clinical_data/designs.txt", sep="|", low_memory=False)
conditions = pd.read_csv("clinical_data/conditions.txt", sep="|", low_memory=False)
eligibilities = pd.read_csv("clinical_data/eligibilities.txt", sep="|", low_memory=False)
design_outcomes = pd.read_csv("clinical_data/design_outcomes.txt", sep="|", low_memory=False)


In [16]:
# Aggregate outcome types and condition names
condition_map = conditions.groupby("nct_id")["name"].apply(lambda x: "|".join(set(x))).reset_index()
outcome_map = design_outcomes.groupby("nct_id")["outcome_type"].apply(lambda x: "|".join(set(x))).reset_index()

# Merge tables
df = (
    studies[["nct_id", "phase", "study_type", "enrollment", "results_first_submitted_date"]]
    .merge(designs[["nct_id", "allocation", "masking"]], on="nct_id", how="left")
    .merge(sponsors[["nct_id", "agency_class"]], on="nct_id", how="left")
    .merge(interventions[["nct_id", "intervention_type"]], on="nct_id", how="left")
    .merge(condition_map, on="nct_id", how="left")
    .merge(outcome_map, on="nct_id", how="left")
    .merge(eligibilities[["nct_id", "gender", "minimum_age", "maximum_age", "healthy_volunteers"]], on="nct_id", how="left")
)

# check if outcome_type is present
print('Columns after merge:', df.columns.tolist())
print('Sample rows after merge:')
print(df.head())

df["trial_success"] = df["results_first_submitted_date"].notnull().astype(int)

# Parse age fields
df["min_age_num"] = df["minimum_age"].str.extract(r"(\d+)").astype(float)
df["max_age_num"] = df["maximum_age"].str.extract(r"(\d+)").astype(float)
df["age_range"] = df["max_age_num"] - df["min_age_num"]
df["gender_all"] = (df["gender"] == "ALL").astype(int)
df["gender_female_only"] = (df["gender"] == "FEMALE").astype(int)
df["gender_male_only"] = (df["gender"] == "MALE").astype(int)

# Create has_primary_outcome BEFORE dummification or dropping
if 'outcome_type' in df.columns:
    df["has_primary_outcome"] = df["outcome_type"].str.contains("primary", case=False, na=False).astype(int)
    # Dummify outcome_type
    all_outcome_types = df["outcome_type"].str.get_dummies(sep="|")
    df = pd.concat([df, all_outcome_types], axis=1)
    # Only drop outcome_type after all features that need it are created
    df = df.drop("outcome_type", axis=1)


Columns after merge: ['nct_id', 'phase', 'study_type', 'enrollment', 'results_first_submitted_date', 'allocation', 'masking', 'agency_class', 'intervention_type', 'name', 'outcome_type', 'gender', 'minimum_age', 'maximum_age', 'healthy_volunteers']
Sample rows after merge:
        nct_id   phase      study_type  enrollment  \
0  NCT02556710  PHASE3  INTERVENTIONAL       480.0   
1  NCT02556710  PHASE3  INTERVENTIONAL       480.0   
2  NCT03826823     NaN  INTERVENTIONAL        89.0   
3  NCT02742103  PHASE2  INTERVENTIONAL        83.0   
4  NCT02742103  PHASE2  INTERVENTIONAL        83.0   

  results_first_submitted_date  allocation masking agency_class  \
0                   2022-07-12  RANDOMIZED  DOUBLE     INDUSTRY   
1                   2022-07-12  RANDOMIZED  DOUBLE     INDUSTRY   
2                          NaN         NaN    NONE        OTHER   
3                   2020-05-26  RANDOMIZED  DOUBLE     INDUSTRY   
4                   2020-05-26  RANDOMIZED  DOUBLE     INDUSTRY   

## EDA

In [28]:
# Missing Values
missing = df.isnull().mean().sort_values(ascending=False)
print("Missing value ratio per column:\n", missing)
plt.figure(figsize=(10, 6))
sns.barplot(x=missing[missing > 0], y=missing[missing > 0].index, color='skyblue')
plt.title('Missing Value Ratio per Column')
plt.xlabel('Fraction Missing')
plt.ylabel('Column')
plt.tight_layout()
plt.savefig(os.path.join(viz_dir, "eda_missing_values_barplot.png"))
plt.close()

# Target Balance
plt.figure(figsize=(6,4))
sns.countplot(x='trial_success', data=df)
plt.title('Trial Success Distribution')
plt.xticks([0, 1], ['Failure', 'Success'])
plt.tight_layout()
plt.savefig(os.path.join(viz_dir, "eda_trial_success_distribution.png"))
plt.close()

# Phase Distribution
plt.figure(figsize=(8,4))
sns.countplot(x='phase', hue='trial_success', data=df)
plt.title('Trial Phase vs Success')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(viz_dir, "eda_phase_vs_success.png"))
plt.close()

# Sponsor Type
plt.figure(figsize=(8,4))
sns.countplot(x='agency_class', hue='trial_success', data=df)
plt.title('Sponsor Type vs Success')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(viz_dir, "eda_sponsor_type_vs_success.png"))
plt.close()

# Intervention Type
plt.figure(figsize=(8,4))
sns.countplot(x='intervention_type', hue='trial_success', data=df)
plt.title('Intervention Type vs Success')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(viz_dir, "eda_intervention_type_vs_success.png"))
plt.close()

# Outcome Type
if 'outcome_type' in df.columns:
    plt.figure(figsize=(8,4))
    sns.countplot(x='outcome_type', hue='trial_success', data=df)
    plt.title('Outcome Type vs Success')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(viz_dir, "eda_outcome_type_vs_success.png"))
    plt.close()

# Enrollment Distribution (Log scale)
df['enrollment'] = pd.to_numeric(df['enrollment'], errors='coerce')
plt.figure(figsize=(8,4))
sns.histplot(df['enrollment'], bins=100, log_scale=True)
plt.title('Log-Scaled Enrollment Distribution')
plt.tight_layout()
plt.savefig(os.path.join(viz_dir, "eda_log_enrollment_distribution.png"))
plt.close()

# Age Range Distribution
df['min_age_num'] = df['minimum_age'].str.extract(r'(\\d+)').astype(float)
df['max_age_num'] = df['maximum_age'].str.extract(r'(\\d+)').astype(float)
plt.figure(figsize=(8,4))
sns.histplot(df['max_age_num'] - df['min_age_num'])
plt.title('Participant Age Range')
plt.xlabel('Age Range (years)')
plt.tight_layout()
plt.savefig(os.path.join(viz_dir, "eda_age_range_distribution.png"))
plt.close()

# Gender Breakdown
plt.figure(figsize=(8,4))
sns.countplot(x='gender', hue='trial_success', data=df)
plt.title('Gender Eligibility vs Success')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(viz_dir, "eda_gender_vs_success.png"))
plt.close()

# Heatmap of Correlations (numeric only)
numeric = df[['enrollment', 'min_age_num', 'max_age_num']].dropna()
corr = numeric.corr()
plt.figure(figsize=(6,5))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.tight_layout()
plt.savefig(os.path.join(viz_dir, "eda_correlation_heatmap.png"))
plt.close()

Missing value ratio per column:
 results_first_submitted_date    0.838614
maximum_age                     0.468809
minimum_age                     0.061095
healthy_volunteers              0.015334
name                            0.000607
study_type                      0.000581
min_age_num                     0.000000
primary                         0.000000
other                           0.000000
has_primary_outcome             0.000000
gender_male_only                0.000000
gender_female_only              0.000000
gender_all                      0.000000
age_range                       0.000000
max_age_num                     0.000000
nct_id                          0.000000
trial_success                   0.000000
phase                           0.000000
gender                          0.000000
intervention_type               0.000000
agency_class                    0.000000
masking                         0.000000
allocation                      0.000000
enrollment              

/Users/zachmumbauer/code/gen_ai/.venv/lib/python3.11/site-packages/seaborn/matrix.py:202: RuntimeWarning: All-NaN slice encountered
  vmin = np.nanmin(calc_data)
/Users/zachmumbauer/code/gen_ai/.venv/lib/python3.11/site-packages/seaborn/matrix.py:207: RuntimeWarning: All-NaN slice encountered
  vmax = np.nanmax(calc_data)


In [17]:
# Analyze missing values
missing_percentage = df.isnull().sum() / len(df) * 100
missing_info = pd.DataFrame({'column': missing_percentage.index, 'percentage': missing_percentage.values})
missing_info = missing_info.sort_values(by='percentage', ascending=False)
print("Missing Values Percentage:")
print(missing_info[missing_info['percentage'] > 0])

# Ensure visualization output directory exists
viz_dir = "clinical_visualizations"
os.makedirs(viz_dir, exist_ok=True)

# Plot missing values
plt.figure(figsize=(10, 6))
sns.barplot(data=missing_info[missing_info['percentage'] > 0], x='percentage', y='column', color='skyblue')
plt.title("Missing Value Percentage by Column")
plt.xlabel("Percent Missing")
plt.ylabel("Column")
plt.tight_layout()
plt.savefig(os.path.join(viz_dir, "missing_values_barplot.png"))
plt.close()


Missing Values Percentage:
                          column  percentage
4   results_first_submitted_date   83.861409
1                          phase   54.006581
17                     age_range   48.815198
12                   maximum_age   46.880946
16                   max_age_num   46.880946
5                     allocation   31.616661
6                        masking   17.544515
15                   min_age_num    6.109521
11                   minimum_age    6.109521
8              intervention_type    5.740141
13            healthy_volunteers    1.533373
3                     enrollment    0.926235
10                        gender    0.108348
9                           name    0.060720
7                   agency_class    0.058127
2                     study_type    0.058127


In [18]:
# Impute missing values
# Numerical columns: Impute with median
for col in ["enrollment", "min_age_num", "max_age_num", "age_range"]:
    df[col] = df[col].fillna(df[col].median())

# Categorical columns: Impute with mode
for col in ["phase", "allocation", "masking", "agency_class", "intervention_type", "gender", "outcome_type"]:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mode()[0])
        if df[col].dtype == 'object':
            df[col] = df[col].infer_objects(copy=False)


## Preprocessing for modeling

In [23]:
outcome_type_cols = [col for col in df.columns if col.startswith('outcome_type_') and col != 'outcome_type']
# Only include columns that actually exist in df
model_features = [
    "phase", "study_type", "allocation", "masking", "agency_class",
    "intervention_type", "enrollment", "age_range",
    "healthy_volunteers_bool", "has_primary_outcome",
    "gender_all", "gender_female_only", "gender_male_only", "trial_success"
] + outcome_type_cols
model_features = [col for col in model_features if col in df.columns]
model_df = df[model_features].copy() # Use .copy() to avoid SettingWithCopyWarning

# One-hot encode categoricals
categorical_cols = ["phase", "study_type", "allocation", "masking", "agency_class", "intervention_type"]
categorical_cols = [col for col in categorical_cols if col in model_df.columns]
print("Columns in model_df before get_dummies:", model_df.columns.tolist())
print("Categorical columns to encode:", categorical_cols)
model_df = pd.get_dummies(model_df, columns=categorical_cols, drop_first=True)

# No need for further preprocessing, as all features are now numeric
X = model_df.drop("trial_success", axis=1)
y = model_df["trial_success"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Columns in model_df before get_dummies: ['phase', 'study_type', 'allocation', 'masking', 'agency_class', 'intervention_type', 'enrollment', 'age_range', 'has_primary_outcome', 'gender_all', 'gender_female_only', 'gender_male_only', 'trial_success']
Categorical columns to encode: ['phase', 'study_type', 'allocation', 'masking', 'agency_class', 'intervention_type']


In [25]:
# Train models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(),
    'XGBoost': XGBClassifier(eval_metric='logloss')
}

for name, model in models.items():
    if name == 'Logistic Regression':
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    auc = roc_auc_score(y_test, y_pred)
    print(f"Model: {name}")
    print(f"AUC: {auc:.4f}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print("-" * 40)


Model: Logistic Regression
AUC: 0.5222
              precision    recall  f1-score   support

           0       0.84      0.99      0.91    265017
           1       0.58      0.05      0.09     51188

    accuracy                           0.84    316205
   macro avg       0.71      0.52      0.50    316205
weighted avg       0.80      0.84      0.78    316205

[[263095   1922]
 [ 48547   2641]]
----------------------------------------
Model: Random Forest
AUC: 0.8049
              precision    recall  f1-score   support

           0       0.93      0.96      0.95    265017
           1       0.78      0.65      0.71     51188

    accuracy                           0.91    316205
   macro avg       0.86      0.80      0.83    316205
weighted avg       0.91      0.91      0.91    316205

[[255562   9455]
 [ 18145  33043]]
----------------------------------------


/Users/zachmumbauer/code/gen_ai/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [02:46:30] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Model: XGBoost
AUC: 0.6084
              precision    recall  f1-score   support

           0       0.87      0.97      0.92    265017
           1       0.63      0.24      0.35     51188

    accuracy                           0.85    316205
   macro avg       0.75      0.61      0.64    316205
weighted avg       0.83      0.85      0.83    316205

[[257567   7450]
 [ 38648  12540]]
----------------------------------------


In [1]:
# SHAP values for XGBoost
xgb_model = models['XGBoost']
X_train_float = X_train.astype(float)
X_test_float = X_test.astype(float)
explainer = shap.Explainer(xgb_model, X_train_float)
shap_values = explainer(X_test_float)

shap.summary_plot(shap_values, X_test_float, plot_type='bar', show=False)
plt.tight_layout()
plt.savefig(os.path.join(viz_dir, "shap_summary_bar.png"))
plt.close()


NameError: name 'models' is not defined